In [1]:
!pip install geemap

import ee
import geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 21.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 KB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.6/99.6 KB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 KB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 KB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 KB 4.4 

In [2]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=dCmUQBhZEomb85apsfOCHSNf8pRhHQGoFnpLCs0ZH-0&tc=FIc25NLmLiJEXMXrn-sMp35_NYHBScSmutqsHREJU_I&cc=_HZcBnd0gj0bE-Rg2_s90UtPu2rnuDThdHpzd_-FJHM

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWtgzh6s5FDFSNabrwWf4mTHA8K4KPkt6n4NH94_iHZO0-7nPGU2vgOfutY

Successfully saved authorization token.


In [3]:
# Takes an image collection and returns a collection with monthly means
def col2monthly(col):
    
    def monthly(m):
        return (col.filter(ee.Filter.calendarRange(y_start, y_end, 'year'))
                .filter(ee.Filter.calendarRange(m, m, 'month')).mean()
                .set('month', m)
                .set('ID', ee.Number(m).format('%02d'))
                )

    
    months = ee.List.sequence(1, 12)
    col_m = ee.ImageCollection.fromImages(months.map(monthly).flatten())
    
    return col_m

def grid2scale(img):
    return img.reproject(crs='EPSG:4326', scale=scale)

# Parameters for computation
y_start = 2001
y_end = 2020
scale = 11100


In [4]:
# Define climate variables to use for clustering
# gives mm/h
p = col2monthly(ee.ImageCollection("NASA/GPM_L3/IMERG_MONTHLY_V06").select('precipitation')).map(grid2scale)

# gives K
t = col2monthly(ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY").select('temperature_2m')).map(grid2scale)

# Define region of interest (roi)
roi = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017").filter(ee.Filter.eq('country_co', 'BR'))

In [5]:
p.getInfo()

{'type': 'ImageCollection',
 'bands': [],
 'features': [{'type': 'Image',
   'bands': [{'id': 'precipitation',
     'data_type': {'type': 'PixelType', 'precision': 'float'},
     'crs': 'EPSG:4326',
     'crs_transform': [0.09971299653726688,
      0,
      0,
      0,
      -0.09971299653726688,
      0]}],
   'properties': {'month': 1, 'ID': '01', 'system:index': '0'}},
  {'type': 'Image',
   'bands': [{'id': 'precipitation',
     'data_type': {'type': 'PixelType', 'precision': 'float'},
     'crs': 'EPSG:4326',
     'crs_transform': [0.09971299653726688,
      0,
      0,
      0,
      -0.09971299653726688,
      0]}],
   'properties': {'month': 2, 'ID': '02', 'system:index': '1'}},
  {'type': 'Image',
   'bands': [{'id': 'precipitation',
     'data_type': {'type': 'PixelType', 'precision': 'float'},
     'crs': 'EPSG:4326',
     'crs_transform': [0.09971299653726688,
      0,
      0,
      0,
      -0.09971299653726688,
      0]}],
   'properties': {'month': 3, 'ID': '03', 'syste

In [6]:
# Let's get the monthly data and calculate the following inputs:
# annual mean monthly precipitation (P_ann),
# annual mean LST (T_ann),
# highest and lowest monthly precipitation (P_max, P_min)
# highest and lowest monthly temperature (T_max, T_min)

p_mean = p.mean().multiply(24*30).rename('p_avg')
p_max = p.max().multiply(24*30).rename('p_max')
p_min = p.min().multiply(24*30).rename('p_min')

t_mean = t.mean().subtract(273).rename('t_avg')
t_max = t.max().subtract(273).rename('t_max')
t_min = t.min().subtract(273).rename('t_min')

img_input = p_mean.addBands([p_max, p_min, t_mean, t_max, t_min]) # Lista com tudo

In [7]:
Map = geemap.Map()

# Display the mean P and T
Map.addLayer(img_input, {}, 'climate_vars')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [8]:
import geemap.colormaps as gcm

cmp = gcm.palettes
parvis = {'min': 15, 'max': 30, 'palette': cmp.viridis_r}

lyrname = 'Average monthly temperature'
Map.addLayer(t_mean, parvis, lyrname)
Map.add_colorbar(parvis, label = None, discrete=False, orientation='vertical', layer_name = lyrname)

In [9]:
# Make the training dataset.
training = img_input.sample(
    region = roi,
    scale = scale,
    numPixels = 10000
)

# Instantiate the clusterer and train it.
clusterer = ee.Clusterer.wekaKMeans(9).train(training)

# Cluster the input using the trained clusterer.
result = img_input.cluster(clusterer).clip(roi)

In [10]:
Map = geemap.Map(center=[-14, -55], zoom=4)

# Koppen climate zones
czns = (ee.FeatureCollection('projects/et-brasil/assets/Climate_Koppen_Brasil')
    .reduceToImage(properties=['GRIDCODE'], reducer=ee.Reducer.first())
    )

# Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'Climate clusters K-means')
Map.addLayer(czns.randomVisualizer(), {}, 'Koppen Climate Zones')
Map

Map(center=[-14, -55], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [11]:
Map = geemap.Map(center=[-14, -55], zoom=4)

l_lyr = geemap.ee_tile_layer(result.randomVisualizer(), {}, 'Climate clusters K-means')
r_lyr = geemap.ee_tile_layer(czns.randomVisualizer(), {}, 'Koppen Climate Zones')

Map.split_map(left_layer=l_lyr, right_layer=r_lyr)
Map

Map(center=[-14, -55], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [12]:
from google.colab import output
output.enable_custom_widget_manager()

geemap.linked_maps(rows=1, cols=2, ee_objects=[result.randomVisualizer(), czns.randomVisualizer()], 
                   center=[-14, -55], zoom=4,
                   vis_params=[{}, {}], labels=['1', '2'])

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [15]:
task_config = {
        'image': result,
        'description': 'Climate_KMeans',
        'folder': 'GEE_intro',
        'scale': scale,
        'region': roi.geometry(),
        'maxPixels': 1e9
        }
task = ee.batch.Export.image.toDrive(**task_config)
task.start()